<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import spacy
yelp = pd.read_json('./data/review_sample.json', lines=True)

In [2]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [3]:
nlp = spacy.load("en_core_web_sm")
def tokenize(doc):
    text = nlp(doc)
    return [token.lemma_ for token in text if (token.is_stop != True) and (token.is_punct != True)]

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [4]:
words = yelp['text'].tolist()
ratings = yelp['stars'].to_numpy()

In [5]:
vect = TfidfVectorizer(tokenizer=tokenize)
dtm = vect.fit_transform(words)
dtm = pd.DataFrame(dtm.todense(), columns = vect.get_feature_names())

In [6]:
fake = ['This place is awful!']
nn = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=10, p=2, radius=1.0)

In [7]:
madeup = vect.transform(fake)
locations = nn.kneighbors(madeup.todense())[1]
for location in locations:
    print(yelp['text'][location])

5209    Food is awful. Very disappointing because this...
3313    My husband has been trying to reach Century Li...
6827    Awful, rude service. They act like they don't ...
457     Awful service and unfriendly staff. \n\nWe wer...
8475    God awful place. Dirty and unorganized. People...
4619    Awful experience. Screaming kids. Long wait. D...
6205    What a awful meal and way overpriced. Service ...
962     Awful. Absolutely awful.\n\nWent here with som...
8689    Rico, the bartender, was awful. He was rude, f...
9587    Other than the pricing, this company is awful....
Name: text, dtype: object


## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [8]:
rfc = RandomForestClassifier()
pipe = Pipeline([('vect', vect), ('clf', rfc)])
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

In [9]:
# grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1)
# grid_search.fit(words, ratings)

In [10]:
# grid_search.predict(madeup)

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [11]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

C:\Users\Michael\Anaconda3\envs\NLP\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Learn the vocubalary of the yelp data:

In [12]:
somewords = yelp['text'].apply(tokenize)
id2word = Dictionary(somewords)

Create a bag of words representation of the entire corpus

In [13]:
corpus = [id2word.doc2bow(text) for text in somewords]

Your LDA model should be ready for estimation: 

In [14]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 3
                  )

Create 1-2 visualizations of the results

In [15]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [17]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

C:\Users\Michael\Anaconda3\envs\NLP\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.001109 -0.004267       1        1  33.572098
2     -0.004682  0.001274       2        1  33.478012
0      0.003574  0.002993       3        1  32.949886, topic_info=     Category         Freq        Term         Total  loglift  logprob
321   Default  3052.000000          \n   3052.000000  30.0000  30.0000
147   Default  4689.000000        food   4689.000000  29.0000  29.0000
208   Default  4092.000000       great   4092.000000  28.0000  28.0000
434   Default  1461.000000         day   1461.000000  27.0000  27.0000
83    Default  2950.000000          go   2950.000000  26.0000  26.0000
40    Default  2456.000000         try   2456.000000  25.0000  25.0000
98    Default   998.000000       night    998.000000  24.0000  24.0000
620   Default  1358.000000       drink   1358.000000  23.0000  23.0000
39    Default  4100.000000        time   4100.000000  22.0000  22.0000
509   Default   730.000000      flavor    730.000000  21.0000  21.0000
97    Default  1708.000000        nice   1708.000000  20.0000  20.0000
43    Default  3743.000000        come   3743.000000  19.0000  19.0000
327   Default   619.000000     awesome    619.000000  18.0000  18.0000
129   Default  1319.000000      people   1319.000000  17.0000  17.0000
82    Default  2775.000000         get   2775.000000  16.0000  16.0000
1000  Default  1052.000000         new   1052.000000  15.0000  15.0000
506   Default   896.000000        well    896.000000  14.0000  14.0000
64    Default  1001.000000         bar   1001.000000  13.0000  13.0000
705   Default  1091.000000       taste   1091.000000  12.0000  12.0000
52    Default  1495.000000       staff   1495.000000  11.0000  11.0000
692   Default  2117.000000        love   2117.000000  10.0000  10.0000
879   Default   841.000000       clean    841.000000   9.0000   9.0000
417   Default  1447.000000         ask   1447.000000   8.0000   8.0000
55    Default  8126.000000        \n\n   8126.000000   7.0000   7.0000
24    Default  3625.000000        like   3625.000000   6.0000   6.0000
355   Default  1534.000000        find   1534.000000   5.0000   5.0000
301   Default  1744.000000        want   1744.000000   4.0000   4.0000
150   Default  1052.000000         fry   1052.000000   3.0000   3.0000
554   Default  1032.000000      friend   1032.000000   2.0000   2.0000
35    Default  1481.000000         say   1481.000000   1.0000   1.0000
...       ...          ...         ...           ...      ...      ...
140    Topic3   335.458893         bit    825.082397   0.2102  -6.2377
135    Topic3   134.558289          15    313.717621   0.2637  -7.1512
163    Topic3   368.751373        star    921.279541   0.1945  -6.1431
96     Topic3   514.022034        need   1321.115112   0.1662  -5.8109
139    Topic3   459.425934         bad   1172.353149   0.1734  -5.9232
92     Topic3   301.312469        long    746.376587   0.2031  -6.3450
0      Topic3  4465.286133              13750.311523  -0.0145  -3.6491
343    Topic3   528.875854  definitely   1390.878174   0.1432  -5.7824
392    Topic3   539.946289        take   1430.044434   0.1362  -5.7617
688    Topic3   294.998871        home    738.164795   0.1930  -6.3662
367    Topic3   513.708679      little   1359.880615   0.1367  -5.8115
51     Topic3  1168.656738     service   3411.969727   0.0387  -4.9896
705    Topic3   414.245636       taste   1091.765991   0.1411  -6.0267
46     Topic3  1825.335815        good   6025.385742  -0.0840  -4.5437
24     Topic3  1168.136963        like   3625.932861  -0.0225  -4.9900
55     Topic3  2314.163086        \n\n   8126.566406  -0.1459  -4.3064
39     Topic3  1280.891846        time   4100.708008  -0.0534  -4.8979
373    Topic3  1052.994263       order   3259.851562  -0.0199  -5.0938
131    Topic3  1415.940186       place   5044.640625  -0.1604  -4.7976
1      Topic3   691.482300           $   2161.095215  -0.0294  -5.

The yelp reviews are primarily concerned with quality of food and quality of service as topics. In addition, there appears to be a third, less-important topic abou the general ambiance of the restaurant. This suggests a divide between those who primarily focus on the food as food, and those who focus on the dining experience.

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)